In [7]:
# -*- coding: utf-8 -*
from math import exp


# 计算z
def calculate_z(weights, inputs):
    z = weights[-1]  # 偏差b  # z = x₁w₁ + x₂w₂ + ... + b
    for i in range(len(weights) - 1):
        z += weights[i] * inputs[i]
    return z


# 激活函数Sigmoid:  σ(z) =  1 / (1 + e^(-z))
def active(z):
    return 1.0 / (1.0 + exp(-z))


# 正向传播过程
def forward_pass(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            z = calculate_z(neuron['weights'], inputs)  # 计算z
            neuron['output'] = active(z)  # 代入激活函数得到输出 并保存到 output key中
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs


# sigmoid函数的导数 : 𝑑σ/𝑑z = σ(1-σ)
def active_derivative(output):
    return output * (1.0 - output)


# 反向传播过程
def back_pass(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]  # 从输出层开始
        errors = []
        if i != len(network) - 1:  # 如果非输出层
            for j in range(len(layer)):  # 遍历所有神经元
                error = 0.0
                # 计算 ∂Etotal/∂a = ∂Eo₁/∂ah₁ + ∂Eo₂/∂ah₁ ...
                for neuron in network[i + 1]:  # 下一层的神经元
                    error += (neuron['weights'][j] * neuron['delta'])  # 反向传播
                errors.append(error)
        else:  # 如果是输出层
            for j in range(len(layer)):
                neuron = layer[j]  # 遍历输出的神经元
                errors.append(neuron['output'] - expected[j])  # -(target - output) 计算∂E/∂a

        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * active_derivative(neuron['output'])  # δ = ∂E/∂zⱼ = ∂E/∂yⱼ * daⱼ/dzⱼ


# 更新参数
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]  # 去掉数据集中最后的类别标签
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]  # 如果不是输入层，则更新该层的输入为上一层的输出
        for neuron in network[i]:
            for j in range(len(inputs)):  # 入参的数量也就是权值参数的数量
                neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]  # wⱼ  = wⱼ  - α * δ * xⱼ
                neuron['weights'][-1] -= l_rate * neuron['delta'] * 1  # 同时更新了偏差b,如果更新了偏差，结果会更好


def total_error(outputs, expected):
    sum_error = 0.0
    for i in range(len(expected)):
        sum_error += (expected[i] - outputs[i]) ** 2
    return sum_error / 2.0


def test():
    network = [[{'weights': [1, -2, 1]},
                {'weights': [-1, 1, 1]}],
               [{'weights': [2, -2, 1]},
                {'weights': [-2, -1, 1]}]]

    dataset = [[1, -1, None]]
    n_inputs = len(dataset[0]) - 1
    expected = [0.01, 0.99]
    l_rate = 0.5

    for times in range(10000):
        sum_error = 0.0
        for row in dataset:
            outputs = forward_pass(network, row)
            sum_error += total_error(outputs, expected)
            back_pass(network, expected)
            update_weights(network, row, l_rate)
        print('>times=%d, lrate=%.3f, error=%.3f' % (times, l_rate, sum_error))

    outputs = forward_pass(network, dataset[0])
    print(outputs)


if __name__ == '__main__':
    test()

>times=0, lrate=0.500, error=0.705
>times=1, lrate=0.500, error=0.670
>times=2, lrate=0.500, error=0.631
>times=3, lrate=0.500, error=0.588
>times=4, lrate=0.500, error=0.544
>times=5, lrate=0.500, error=0.500
>times=6, lrate=0.500, error=0.457
>times=7, lrate=0.500, error=0.413
>times=8, lrate=0.500, error=0.369
>times=9, lrate=0.500, error=0.322
>times=10, lrate=0.500, error=0.273
>times=11, lrate=0.500, error=0.225
>times=12, lrate=0.500, error=0.181
>times=13, lrate=0.500, error=0.144
>times=14, lrate=0.500, error=0.115
>times=15, lrate=0.500, error=0.094
>times=16, lrate=0.500, error=0.078
>times=17, lrate=0.500, error=0.066
>times=18, lrate=0.500, error=0.057
>times=19, lrate=0.500, error=0.050
>times=20, lrate=0.500, error=0.044
>times=21, lrate=0.500, error=0.040
>times=22, lrate=0.500, error=0.036
>times=23, lrate=0.500, error=0.033
>times=24, lrate=0.500, error=0.030
>times=25, lrate=0.500, error=0.028
>times=26, lrate=0.500, error=0.026
>times=27, lrate=0.500, error=0.024
>t